In [8]:
from glob import glob
import numpy as np
import cv2
from gi.repository import Vips
import matplotlib
from matplotlib import pyplot as plt
from skimage import exposure
from scipy.misc import bytescale
import pvl
import os.path
import numpy
import shutil
from math import floor
import pyproj

%matplotlib inline

In [9]:
def imshow(img):
    if len(img.shape) == 2 or img.shape[2] == 1:
        img = np.dstack([img, img, img])
    b, g, r = cv2.split(img)
    rgb_img = cv2.merge([r, g, b])
    plt.imshow(rgb_img)
    plt.show()

In [10]:
def read_vips(filename):
    return Vips.Image.new_from_file(filename)

def extract_gray(image, offset_x, offset_y, size_x=256, size_y=256):
    subimage = image.extract_area(offset_x, offset_y, size_x, size_y)
    mat = np.fromstring(subimage.write_to_memory(), np.int16).view(np.int16)
    return np.uint8(mat.reshape(size_y, size_x) / 8)

def extract_bgr(image, offset_x, offset_y, size_x=256, size_y=256):
    subimage = image.extract_area(offset_x, offset_y, size_x, size_y)
    channels = [1, 2, 4]
    mat = np.fromstring(subimage.write_to_memory(), np.int16).view(np.int16)
    return np.uint8(mat.reshape(size_y, size_x, 8) / 8)[:, :, channels]
    
def rescale_bands(image):
    scaled_image = np.zeros(image.shape, dtype=np.float)
    for i in range(3):
        p1, p2 = [(32, 128), (32, 192), (16, 168)][i]
        scaled_image[:, :, i] = exposure.rescale_intensity(image[:, :, i], in_range=(p1, p2))
    return bytescale(scaled_image)

def silent_make_dir(path):
    try:
        os.makedirs(path)
    except FileExistsError:
        pass
    
def silent_delete_dir(path):
    success = False
    if os.path.exists(path):
        while not success:
            try:
                shutil.rmtree(path)
                success = True
            except FileNotFoundError:
                pass
            
def _split(xys):
    if isinstance(xys, tuple):
        x, y = xys
        return x, y
    elif isinstance(xys, np.ndarray):
        xs, ys = xys.T
        return xs, ys
    else:
        raise Exception
            
_EARTH_RADIUS_M = 6378137.0

_WMTS_WIDTH = 2.0 * np.pi * _EARTH_RADIUS_M
_WMTS_OFFSET = -0.5 * _WMTS_WIDTH
WMTS_TILE_SIZE = 256
            
_wms_proj = pyproj.Proj(proj='merc', a=_EARTH_RADIUS_M, b=_EARTH_RADIUS_M)
def lonlat_to_mercator(p):
    lon, lat = _split(p)
    return np.float64(_wms_proj(lon, lat))

In [11]:
def mercator_to_pixel(val):
    return round(val / 0.4 - 0.5)

def read_or_create(x, y, lod, num):
    global output_path
    filename = output_path + "{3}/{0}/{1}_{2}.png".format(lod, x, y, num)
    
    if os.path.exists(filename):
        return cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    else:
#         if lod == 18:
        return numpy.zeros((256, 256, 4), np.uint8)
#         else:
#             return numpy.ones((256, 256, 4), np.uint8) * 255
        
def write(x, y, lod, num, img):
    global output_path
    filename = output_path + "{3}/{0}/{1}_{2}.png".format(lod, x, y, num)
    cv2.imwrite(filename, img)

def process_tif(image, lod, num, lllon, lllat, urlon, urlat, is_colored=False):
    silent_make_dir(output_path + "{0}/{1}/".format(num, lod))
    
    llx_real, lly_real = lonlat_to_mercator((lllon, lllat))
    urx_real, ury_real = lonlat_to_mercator((urlon, urlat))
    
    llx = llx_real - _WMTS_OFFSET
    lly = lly_real - _WMTS_OFFSET
    urx = urx_real - _WMTS_OFFSET
    ury = ury_real - _WMTS_OFFSET
    
    dx = (urx - llx) / image.width
    dy = (ury - lly) / image.height
    
    lly, ury = _WMTS_WIDTH - ury, _WMTS_WIDTH - lly

    img_size_x = image.width
    img_size_y = image.height
    
    tile_width = _WMTS_WIDTH / 2 ** lod
    
    tile_num_from_x = floor(llx / tile_width)
    tile_num_from_y = floor(lly / tile_width)    

    tile_num_to_x = floor(urx / tile_width) + 1
    tile_num_to_y = floor(ury / tile_width) + 1

    for tile_x in range(tile_num_from_x, tile_num_to_x):
        for tile_y in range(tile_num_from_y, tile_num_to_y):
            tile = read_or_create(tile_x, tile_y, lod, num)

            lx = tile_x * tile_width
            rx = (tile_x + 1) * tile_width
            ly = tile_y * tile_width
            ry = (tile_y + 1) * tile_width
            
            lx_p = int((lx - llx) / dx)
            ly_p = int((ly - lly) / dy)
            
            rx_p = int((rx - llx) / dx)
            ry_p = int((ry - lly) / dy)
            
            offset_x = max(0, -lx_p)
            offset_y = max(0, -ly_p)
            
            x_range = min(rx_p - lx_p, img_size_x - lx_p) - offset_x
            y_range = min(ry_p - ly_p, img_size_y - ly_p) - offset_y

            if x_range <= 0 or y_range <= 0:
                continue
            
            if is_colored:
                cur_img = rescale_bands(extract_bgr(image, lx_p + offset_x, ly_p + offset_y, x_range, y_range))
            else:
                cur_img = extract_gray(image, lx_p + offset_x, ly_p + offset_y, x_range, y_range)
            
            cur_tile = numpy.zeros((ry_p - ly_p, rx_p - lx_p, 4), np.uint8)
            if is_colored:
                cur_tile[offset_y:offset_y + y_range, offset_x:offset_x + x_range, 0:3] = cur_img
                cur_tile[offset_y:offset_y + y_range, offset_x:offset_x + x_range, 3] = 255
            else:
                cur_tile[offset_y:offset_y + y_range, offset_x:offset_x + x_range, 0] = cur_img
                cur_tile[offset_y:offset_y + y_range, offset_x:offset_x + x_range, 1] = cur_img
                cur_tile[offset_y:offset_y + y_range, offset_x:offset_x + x_range, 2] = cur_img
                cur_tile[offset_y:offset_y + y_range, offset_x:offset_x + x_range, 3] = 255

            cur_tile = cv2.resize(cur_tile, (256, 256))

            tile += cur_tile
            write(tile_x, tile_y, lod, num, tile)

def build_from(lod, num):
    global output_path
    cur_folder = "{0}{1}/{2}/".format(output_path, num, lod) 
    next_folder = "{0}{1}/{2}/".format(output_path, num, lod + 1) 
    
    silent_delete_dir(cur_folder)
    silent_make_dir(cur_folder)
    
    s = set()
    
    for f in os.listdir(next_folder):
        under_pos = f.index('_')
        dot_pos = f.index('.')
        
        x = int(f[:under_pos])
        y = int(f[under_pos + 1: dot_pos])
        
        s.add((x // 2, y // 2))
        
    for tile_x, tile_y in s:
        new_tile = np.zeros((512, 512, 4), np.uint8)
        
        for dx in range(2):
            for dy in range(2):
                old_tile = read_or_create(tile_x * 2 + dx, tile_y * 2 + dy, lod + 1, num)
                new_tile[dy * 256 : (dy + 1) * 256, dx * 256 : (dx + 1) * 256] = old_tile
                
        new_tile = cv2.resize(new_tile, (256, 256))
        write(tile_x, tile_y, lod, num, new_tile)

In [13]:
def process_tiled_img(info_name, num, lod, is_colored=False):
    info = pvl.load(dir_path + info_name)

    r1c1 = Vips.Image.new_from_file(dir_path + info['TILE_1']['filename'])
    r1c2 = Vips.Image.new_from_file(dir_path + info['TILE_2']['filename'])
    r2c1 = Vips.Image.new_from_file(dir_path + info['TILE_3']['filename'])
    r2c2 = Vips.Image.new_from_file(dir_path + info['TILE_4']['filename'])

    upper = r1c1.join(r1c2, Vips.Direction.HORIZONTAL)
    lower = r2c1.join(r2c2, Vips.Direction.HORIZONTAL)
    overall = upper.join(lower, Vips.Direction.VERTICAL) 

    process_tif(overall, lod, num, 
                 info['TILE_3']['LLLon'], info['TILE_3']['LLLat'], 
                 info['TILE_2']['URLon'], info['TILE_2']['URLat'], is_colored)
    for i in range(lod - 1, 0, -1):
        build_from(i, num)
   
output_path = "./data/output/"
silent_delete_dir(output_path)

dir_path = "./data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_PAN/"
process_tiled_img('14OCT06063756-P2AS-053950035020_01_P001.TIL', 0, 18)
process_tiled_img('14OCT06063658-P2AS-053950035020_01_P001.TIL', 1, 18)

dir_path = "./data/OR2A Stereo, Muscat, Oman, 40cm_053950035020/053950035020_01_P001_MUL/"
process_tiled_img('14OCT06063756-M2AS-053950035020_01_P001.TIL', 2, 18, True)
process_tiled_img('14OCT06063658-M2AS-053950035020_01_P001.TIL', 3, 18, True)